In [2]:
!nvidia-smi

Tue Apr  6 09:23:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.06    Driver Version: 450.51.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install -q torch==1.8.1+cu111 torchvision==0.9.1+cu111 -f https://download.pytorch.org/whl/torch_stable.html

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.2.7 requires torch<1.8,>=1.7.0, but you have torch 1.8.1+cu111 which is incompatible.
fastai 2.2.7 requires torchvision<0.9,>=0.8, but you have torchvision 0.9.1+cu111 which is incompatible.


In [6]:
!pip install -q quickvision==0.2.0

In [8]:
!pip uninstall torchtext -y
!pip uninstall torchaudio -y

Found existing installation: torchaudio 0.8.1
Uninstalling torchaudio-0.8.1:
  Successfully uninstalled torchaudio-0.8.1


In [21]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms as T
from tqdm import tqdm
from quickvision.models.classification import cnn
from quickvision import utils
print(torch.__version__)

1.8.1+cu111


In [22]:
!pwd

/kaggle/working


In [26]:
DATA_DIR = "../input/aptos2019-blindness-detection/"
TRAIN_DIR = "../input/aptos2019-blindness-detection/train_images"
CSV_PATH = "../input/aptos2019-blindness-detection/train.csv"
MODEL_PATH = "./"


LEARNING_RATE = 1e-3
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
NUM_WORKERS = 2
EPOCHS = 2

In [27]:
class RetinopathyDataset(Dataset):
    def __init__(self, image_dir, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.image_dir = image_dir

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # img_name = os.path.join('../input/aptos2019-blindness-detection/train_images',
        #                         self.data.loc[idx, 'id_code'] + '.png')

        img_name = os.path.join(self.image_dir, self.data.loc[idx, 'id_code'] + '.png')

        image = Image.open(img_name)
        image = image.resize((512, 512), resample=Image.BILINEAR)
        label = torch.tensor(self.data.loc[idx, 'diagnosis'], dtype=torch.long)

        if self.transform is not None:
            img = self.transform(image)

        return (img, label)

In [28]:
utils.seed_everything(42)
utils.set_debug_apis(False)

train_trasforms = T.Compose([
    T.ToTensor(),
#     T.Resize((224, 224)),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    # T.ConvertImageDtype(torch.float32),
    # T.CenterCrop((224, 224)),
])

train_dataset = RetinopathyDataset(TRAIN_DIR, CSV_PATH, transform=train_trasforms)
# model = model.create_model("")

train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE,
                      shuffle=False, num_workers=NUM_WORKERS)


for batch_idx, (inputs, target) in enumerate(train_loader):
    print(batch_idx)
#     print(inputs)
    print(target)
    break

0
tensor([2, 4, 1, 0, 0, 4, 0, 2, 2, 1, 0, 2, 0, 3, 1, 0, 2, 0, 0, 2, 2, 0, 1, 2,
        0, 2, 0, 0, 0, 0, 0, 0])


In [29]:
print(torch.cuda.is_available())

True


In [30]:
model = cnn.create_cnn("resnet50", num_classes=5, pretrained=None)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

# # if config.USE_AMP:
# #     from torch.cuda import amp
# #     scaler = amp.GradScaler()


# print(device)
model = model.to("cuda")

for epoch in tqdm(range(EPOCHS)):
    metrics = cnn.train_step(model, train_loader, criterion, device, optimizer)

# Save model every epoch
torch.save(model.state_dict(), MODEL_PATH + f"_{epoch}" + ".pt")

  0%|          | 0/2 [00:00<?, ?it/s]

Batch Train Time: 11.064 (11.064)  Loss:  1.8092 (1.8092)  Top 1 Accuracy: 18.7500 (18.7500)  Top 5 Accuracy: 100.0000 (100.0000)
Batch Train Time: 0.864 (4.005)  Loss:  1.2235 (1.2615)  Top 1 Accuracy: 46.8750 (49.5668)  Top 5 Accuracy: 100.0000 (100.0000)


 50%|█████     | 1/2 [07:38<07:38, 458.43s/it]

Batch Train Time: 3.198 (3.985)  Loss:  1.0995 (1.2526)  Top 1 Accuracy: 50.0000 (50.3550)  Top 5 Accuracy: 100.0000 (100.0000)
Time taken for train step = 458.4207293987274 sec
Batch Train Time: 8.064 (8.064)  Loss:  1.1764 (1.1764)  Top 1 Accuracy: 50.0000 (50.0000)  Top 5 Accuracy: 100.0000 (100.0000)
Batch Train Time: 0.874 (4.010)  Loss:  1.1227 (1.1753)  Top 1 Accuracy: 59.3750 (57.6423)  Top 5 Accuracy: 100.0000 (100.0000)


100%|██████████| 2/2 [15:14<00:00, 457.20s/it]

Batch Train Time: 0.394 (3.964)  Loss:  0.9721 (1.1603)  Top 1 Accuracy: 57.1429 (58.2196)  Top 5 Accuracy: 100.0000 (100.0000)
Time taken for train step = 455.9706521034241 sec
